In [ ]:
# Social exp timeline

# Aeon3
#  2024-01-31 : 2024-02-03 - BAA-1104045 pre solo
#  2024-02-05 : 2024-02-08 - BAA-1104047 pre solo (dominant)
#  2024-02-09 : 2024-02-23 - BAA-1104045, BAA-1104047 social
#  2024-02-25 : 2024-02-28 - BAA-1104045 post solo
#  2024-02-28 : 2024-03-02 - BAA-1104047 post solo

# Aeon4
#  2024-01-31 : 2024-02-03 - BAA-1104048 pre solo (dominant)
#  2024-02-05 : 2024-02-08 - BAA-1104049 pre solo
#  2024-02-09 : 2024-02-23 - BAA-1104048, BAA-1104049 social
#  2024-02-25 : 2024-02-28 - BAA-1104048 post solo
#  2024-02-28 : 2024-03-02 - BAA-1104049 post solo

In [ ]:
"""Imports and settings."""

%load_ext autoreload
%autoreload 2
# %flow mode reactive

import os
import pathlib
from colorsys import hls_to_rgb, rgb_to_hls
from contextlib import contextmanager
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objs as go
import seaborn as sns
from numpy.lib.stride_tricks import as_strided

import aeon
import datajoint as dj
from aeon.schema.schemas import social02
from aeon.dj_pipeline.analysis.block_analysis import *

## Load and Clean Data

In [ ]:
"""Load data."""

# Allow reading (on Unix-like OS) of .pkl files written on Windows
@contextmanager
def set_windows_path_as_posix():
    windows_backup = pathlib.WindowsPath
    try:
        pathlib.WindowsPath = pathlib.PosixPath
        yield
    finally:
        pathlib.WindowsPath = windows_backup

def load_data():
    global \
        roots, \
        blocks_df_2024_01_31, \
        blocks_df_2024_02_14, \
        blocks_df_2024_02_16, \
        blocks_df_2024_02_25, \
        blocks_df_2024_02_29
    global \
        skipped_blocks_2024_01_31, \
        skipped_blocks_2024_02_14, \
        skipped_blocks_2024_02_16, \
        skipped_blocks_2024_02_25, \
        skipped_blocks_2024_02_29
    roots = [
        Path(f"{root_prefix}/aeon/data/raw/AEON3/social0.2"),
        Path(f"{root_prefix}/aeon/data/raw/AEON4/social0.2"),
    ]

    blocks_df_2024_01_31 = pd.read_pickle(
        Path(
            f"{root_prefix}/aeon/code/scratchpad/jai/social02/social02_2024-01-31_2024-02-14.pkl"
        )
    )
    blocks_df_2024_02_14 = pd.read_pickle(
        Path(
            f"{root_prefix}/aeon/code/scratchpad/jai/social02/social02_2024-02-14_2024-02-16.pkl"
        )
    )
    blocks_df_2024_02_16 = pd.read_pickle(
        Path(
            f"{root_prefix}/aeon/code/scratchpad/jai/social02/social02_2024-02-16_2024-02-23.pkl"
        )
    )
    blocks_df_2024_02_25 = pd.read_pickle(
        Path(
            f"{root_prefix}/aeon/code/scratchpad/jai/social02/social02_2024-02-25_2024-02-28.pkl"
        )
    )
    blocks_df_2024_02_29 = pd.read_pickle(
        Path(
            f"{root_prefix}/aeon/code/scratchpad/jai/social02/social02_2024-02-29_2024-03-02.pkl"
        )
    )

    skipped_blocks_2024_01_31 = np.load(
        Path(
            f"{root_prefix}/aeon/code/scratchpad/jai/social02/skipped_blocks_2024-01-31_2024-02-14.npy"
        )
    )
    skipped_blocks_2024_02_14 = np.load(
        Path(
            f"{root_prefix}/aeon/code/scratchpad/jai/social02/skipped_blocks_2024-02-14_2024-02-16.npy"
        )
    )
    skipped_blocks_2024_02_16 = np.load(
        Path(
            f"{root_prefix}/aeon/code/scratchpad/jai/social02/skipped_blocks_2024-02-16_2024-02-23.npy"
        )
    )
    skipped_blocks_2024_02_25 = np.load(
        Path(
            f"{root_prefix}/aeon/code/scratchpad/jai/social02/skipped_blocks_2024-02-25_2024-02-28.npy"
        )
    )
    skipped_blocks_2024_02_29 = np.load(
        Path(
            f"{root_prefix}/aeon/code/scratchpad/jai/social02/skipped_blocks_2024-02-29_2024-03-02.npy"
        )
    )

root_prefix = "Z:" if os.name == "nt" else "/ceph/aeon"
if root_prefix == "/ceph/aeon":
    with set_windows_path_as_posix():
        load_data()
else:
    load_data()

In [ ]:
"""Clean data."""

# Concatenate loaded data
blocks_df = pd.concat(
    [
        blocks_df_2024_01_31,
        blocks_df_2024_02_14,
        blocks_df_2024_02_16,
        blocks_df_2024_02_25,
        blocks_df_2024_02_29,
    ]
)
skipped_blocks = np.concatenate(
    [
        skipped_blocks_2024_01_31,
        skipped_blocks_2024_02_14,
        skipped_blocks_2024_02_16,
        skipped_blocks_2024_02_25,
        skipped_blocks_2024_02_29,
    ]
)
# Clean indices
blocks_df.reset_index(inplace=True, drop=True)
# Remove skipped blocks (blocks with few pellets)
good_blocks_df = blocks_df[~skipped_blocks].reset_index(drop=True)
# Make paths consistent with my ceph network map
good_blocks_df["root"] = (
    good_blocks_df["root"]
    .apply(lambda path: Path(str(path).replace("S:", "Z:")))
    .apply(lambda path: Path(str(path).replace("Z:\\", root_prefix)))
)
good_blocks_df["sleap_model_dir"] = (
    good_blocks_df["sleap_model_dir"]
    .apply(lambda path: Path(str(path).replace("S:", "Z:")))
    .apply(lambda path: Path(str(path).replace("Z:\\", root_prefix)))
)
# Round timestamps to nearest ms
good_blocks_df["start"] = good_blocks_df["start"].apply(
    lambda x: x.round("ms"))
good_blocks_df["end"] = good_blocks_df["end"].apply(lambda x: x.round("ms"))
good_blocks_df["block_duration"].apply(lambda x: x.round("ms"))
# Create df we can iterate through (pandas doesn't like format of "cum_wheel_dist" col for some reason)
good_blocks_df_cp = good_blocks_df.drop(columns=["cum_wheel_dist"])

In [ ]:
"""See percentage of 'good' blocks (> 3 pellets)."""

pct_blocks_foraging = len(good_blocks_df_cp) / len(blocks_df)
print(f"{len(good_blocks_df_cp)=}\n{len(good_blocks_df)=}\n{pct_blocks_foraging=:.3f}")
display(good_blocks_df_cp)

In [ ]:
"""View a particular block."""

b_i = 125
block = good_blocks_df_cp.iloc[b_i]
print(block)
# Example for getting cum_wheel_dist
# good_blocks_df["cum_wheel_dist"].iloc[b_i]  # have to use `iloc`

## Block Plots

In [ ]:
"""Standardize subject colors, patch colors, and markers."""

subject_colors = plotly.colors.qualitative.Plotly
subject_colors_dict = {
    "BAA-1104045": subject_colors[0],
    "BAA-1104047": subject_colors[1],
    "BAA-1104048": subject_colors[2],
    "BAA-1104049": subject_colors[3],
}
patch_colors = plotly.colors.qualitative.Dark2
patch_markers = [
    "circle",
    "bowtie",
    "square",
    "hourglass",
    "diamond",
    "cross",
    "x",
    "triangle",
    "star",
]
patch_markers_symbols = ["●", "⧓", "■", "⧗", "♦", "✖", "×", "▲", "★"]
patch_markers_dict = {
    marker: symbol for marker, symbol in zip(patch_markers, patch_markers_symbols)
}
patch_markers_linestyles = ["solid", "dash", "dot", "dashdot", "longdashdot"]

### 1. Patch stats: mean, next to boxplots of each pellet threshold per patch.

In [ ]:
n_patches = len(block.patch_info)
pellet_info = block.pellet_info.sort_values("time").iloc[
    :-n_patches
]  # drop updates for each patch at new block
display(pellet_info)
display(block.patch_info)

In [ ]:
mean_pellet_info = pd.DataFrame(index=np.arange(
    n_patches), columns=pellet_info.columns)
mean_pellet_info["time"] = block.start
mean_pellet_info["id"] = "mean"
for i, patch in enumerate(block.patch_info.index):
    mean_pellet_info.loc[i, "patch"] = patch
    mean_pellet_info.loc[i, "threshold"] = (
        block.patch_info.loc[patch, "mean"] +
        block.patch_info.loc[patch, "offset"]
    )
    mean_pellet_info.loc[i, "patch"] = patch
pellet_info_plus = pd.concat(
    (mean_pellet_info, pellet_info)).reset_index(drop=True)
pellet_info_plus["norm_time"] = (
    (pellet_info_plus["time"] - pellet_info_plus["time"].iloc[0])
    / (pellet_info_plus["time"].iloc[-1] - pellet_info_plus["time"].iloc[0])
).round(3)

In [ ]:
pellet_info_plus

In [ ]:
box_colors = ["#0A0A0A"] + subject_colors[
    0: len(block.subjects)
]  # subject colors + mean color

fig = px.box(
    pellet_info_plus.sort_values("patch"),
    x="patch",
    y="threshold",
    color="id",
    hover_data=["norm_time"],
    color_discrete_sequence=box_colors,
    # notched=True,
    points="all",
)

fig.update_layout(
    title="Patch Stats: Patch Means and Sampled Threshold Values", 
    xaxis_title="Patch", 
    yaxis_title="Threshold (cm)"
)

fig.show()

### 2. Animal weight: over time, per subject

In [ ]:
weights = aeon.load(block.root, social02.Environment.SubjectWeight, block.start, block.end)
weights

In [ ]:
fig = px.line(
    weights,
    x=weights.index,
    y="weight",
    color="subject_id",
    color_discrete_map=subject_colors_dict,
    markers=True
)

fig.update_traces(line=dict(width=3), marker=dict(size=8))


fig.update_layout(
    title="Weights",
    xaxis_title="Time",
    yaxis_title="Weight (g)",
)

fig.show()

### 3a. Animal position: (x, y) over time, per subject

In [ ]:
"""Get pose data."""

# Change model root path to my network mapped ceph path
social02.CameraTop.Pose._model_root = Path(
    f"{root_prefix}/aeon/data/processed")
pose_df = aeon.load(block.root, social02.CameraTop.Pose, block.start, block.end)
pose_df = pose_df.groupby("class").resample("100ms").first().droplevel("class")
pose_df.index = pose_df.index.round("100ms")
pose_df = social02.CameraTop.Pose.class_int2str(pose_df, block.sleap_model_dir)
# Simplify to centroid only.
centroid_df = pose_df[pose_df["part"] == "centroid"].drop(columns=["part", "part_likelihood"])
# centroid_df["x"], centroid_df["y"] = (
#     centroid_df["x"].astype(np.int32),
#     centroid_df["y"].astype(np.int32),
# )
centroid_df["class"] = centroid_df["class"].astype("string")
centroid_df.sort_index(inplace=True)
display(centroid_df)

In [ ]:
"""Create function for generating arrays of hex color values from a single initial hex color."""


def gen_hex_grad(hex_col, vals, min_l=0.3):
    """Generates an array of hex color values based on a gradient defined by unit-normalized values."""
    # Convert hex to rgb to hls
    h, l, s = rgb_to_hls(
        *[int(hex_col.lstrip("#")[i: i + 2], 16) / 255 for i in (0, 2, 4)]
    )
    grad = np.empty(shape=(len(vals),), dtype="<U10")  # init grad
    for i, val in enumerate(vals):
        cur_l = (l * val) + (
            min_l * (1 - val)
        )  # get cur lightness relative to `hex_col`
        cur_l = max(min(cur_l, l), min_l)  # set min, max bounds
        cur_rgb_col = hls_to_rgb(h, cur_l, s)  # convert to rgb
        cur_hex_col = "#%02x%02x%02x" % tuple(
            int(c * 255) for c in cur_rgb_col
        )  # convert to hex
        grad[i] = cur_hex_col

    return grad

In [ ]:
"""Create position over time scatter plot."""

fig = go.Figure()
for id_i, (id_val, id_grp) in enumerate(centroid_df.groupby("class")):
    norm_time = (
        (id_grp.index - id_grp.index[0]) / (id_grp.index[-1] - id_grp.index[0])
    ).values.round(3)
    colors = gen_hex_grad(subject_colors[id_i], norm_time)
    fig.add_trace(
        go.Scatter(
            x=id_grp["x"],
            y=id_grp["y"],
            mode="markers",
            name=id_val,
            marker={
                # "opacity": norm_time,
                "color": colors,
                "size": 4,
            },
        )
    )
fig.update_layout(
    title="Position Tracking over Time",
    xaxis_title="X Coordinate",
    yaxis_title="Y Coordinate",
)
fig.show()

### 3b. Animal position: (x, y) heatmap, per subject

In [ ]:
def conv2d(arr, kernel):
    """Performs "valid" 2d convolution using numpy `as_strided` and `einsum`"""
    out_shape = tuple(np.subtract(arr.shape, kernel.shape) + 1)
    sub_mat_shape = kernel.shape + out_shape
    # Create "new view" of `arr` as submatrices at which kernel will be applied
    sub_mats = as_strided(arr, shape=sub_mat_shape, strides=(arr.strides * 2))
    out = np.einsum("ij, ijkl -> kl", kernel, sub_mats)
    return out

In [ ]:
"""Create position heatmaps per subject."""

max_x, max_y = centroid_df["x"].max(), centroid_df["y"].max()
for id_i, (id_val, id_grp) in enumerate(centroid_df.groupby("class")):
    # <s Add counts of x,y points to a grid that will be used for heatmap
    img_grid = np.zeros((max_x + 1, max_y + 1))
    points, counts = np.unique(
        id_grp[["x", "y"]].values, return_counts=True, axis=0)
    for point, count in zip(points, counts):
        img_grid[point[0], point[1]] = count
    img_grid /= img_grid.max()  # normalize
    # /s>
    # <s Smooth `img_grid`
    # Mice can go ~450 cm/s, we've downsampled to 10 frames/s, we have 200 px / 1000 cm,
    # so 45 cm/frame ~= 9 px/frame
    win_sz = 9  # in pixels  (ensure odd for centering)
    kernel = np.ones((win_sz, win_sz)) / win_sz**2  # moving avg kernel
    img_grid_p = np.pad(
        img_grid, win_sz // 2, mode="edge"
    )  # pad for full output from convolution
    img_grid_smooth = conv2d(img_grid_p, kernel)
    # /s>
    fig = px.imshow(
        img_grid_smooth.T,
        zmin=0,
        zmax=(img_grid_smooth.max() / 1000),
        x=np.arange(img_grid.shape[0]),
        y=np.arange(img_grid.shape[1]),
        labels=dict(x="X", y="Y", color="Norm Freq / 1e3"),
        aspect="auto",
    )
    fig.update_layout(title=f"Position Heatmap ({id_val})")
    fig.show()

### 3c. Animal position: ethogram, per subject

In [ ]:
# Load metadata
exp_start = pd.Timestamp("2024-01-31")
metadata = (
    aeon.load(block.root, social02.Metadata, exp_start, block.end).iloc[0].metadata
)

In [ ]:
# Set-up
patch_radius, gate_radius = 120, 30
patches = list(block.patch_info.index)
rois = patches + ["Nest", "Gate", "Corridor"]  # ROIs: patches, nest, gate, corridor
roi_colors = plotly.colors.qualitative.Dark2
roi_colors_dict = {roi : roi_c for (roi, roi_c) in zip(rois, roi_colors)}
pos_eth_df = pd.DataFrame(columns=(["Subject"] + rois), index=centroid_df.index)  # df to create eth fig
pos_eth_df["Subject"] = centroid_df["class"]

arena_center = tuple(map(int, tuple(metadata.ActiveRegion.ArenaCenter.values())[0:2]))
arena_inner_radius = int(metadata.ActiveRegion.ArenaInnerRadius)
arena_outer_radius = int(metadata.ActiveRegion.ArenaOuterRadius)

# For each ROI, compute if within ROI
for roi in rois:
    if roi == "Corridor":  # special case for corridor, based on between inner and outer radius
        dist = np.linalg.norm((np.vstack((centroid_df["x"], centroid_df["y"])).T) - arena_center, axis=1)
        pos_eth_df[roi] = (dist >= arena_inner_radius) & (dist <= arena_outer_radius)
    elif roi == "Nest":  # special case for nest, based on 4 corners
        nest_corners = metadata.ActiveRegion.NestRegion.ArrayOfPoint
        nest_br_x, nest_br_y = int(nest_corners[0]["X"]), int(nest_corners[0]["Y"])
        nest_bl_x, nest_bl_y = int(nest_corners[1]["X"]), int(nest_corners[1]["Y"])
        nest_tl_x, nest_tl_y = int(nest_corners[2]["X"]), int(nest_corners[2]["Y"])
        nest_tr_x, nest_tr_y = int(nest_corners[3]["X"]), int(nest_corners[3]["Y"])
        pos_eth_df[roi] = (
            (centroid_df["x"] <= nest_br_x) & (centroid_df["y"] >= nest_br_y)
            & (centroid_df["x"] >= nest_bl_x) & (centroid_df["y"] >= nest_bl_y)
            & (centroid_df["x"] >= nest_tl_x) & (centroid_df["y"] <= nest_tl_y)
            & (centroid_df["x"] <= nest_tr_x) & (centroid_df["y"] <= nest_tr_y)
        )
    else:
        roi_radius = gate_radius if roi =="Gate" else patch_radius
        # Get ROI coords
        roi_x, roi_y = tuple(map(int, tuple(metadata.Devices[roi + "Rfid"].Location.values())[0:2]))
        # Check if in ROI
        dist = np.linalg.norm((np.vstack((centroid_df["x"], centroid_df["y"])).T) - (roi_x, roi_y), axis=1)
        pos_eth_df[roi] = dist < roi_radius

In [ ]:
# Figure

# Melt df to a single "Loc" column that contains loc for current time (row)
pos_eth_df = pos_eth_df.iloc[::100]  # downsample to 10s bins
melted_df = pos_eth_df.reset_index().melt(id_vars=["time", "Subject"], var_name="Loc", value_name="Val")
melted_df = melted_df[melted_df["Val"]]

# Plot using Plotly Express
fig = px.scatter(
    melted_df, 
    x="time", 
    y="Subject", 
    color="Loc",
    color_discrete_map=roi_colors_dict,
)

fig.update_layout(
    title="Position Ethogram",
    xaxis_title="Time",
    yaxis_title="Subject",
    width=1000,
    height=250,
    yaxis=dict(
        categoryorder="total ascending",
        categoryarray=sorted(melted_df["Subject"].unique()),
        tickmode="array",
        tickvals=sorted(melted_df["Subject"].unique()),
        ticktext=sorted(melted_df["Subject"].unique()),
    ),
)

fig.show()

### 4. Animal behavior: ethogram, per subject

### 5a. Cumulative pellet count over time, per subject, markered by patch

In [ ]:
def cumsum_helper(group):
    group["counter"] = np.arange(len(group)) + 1
    return group

cum_pel_ct = (
    pellet_info.groupby("id", group_keys=False)
    .apply(cumsum_helper)
    .reset_index(drop=True)
)
cum_pel_ct = cum_pel_ct.merge(
    mean_pellet_info[["patch", "threshold"]].rename(
        columns={"threshold": "mean_thresh"}
    ),
    on="patch",
    how="left",
)
cum_pel_ct["patch_label"] = cum_pel_ct.apply(
    lambda row: f"{row['patch']} μ: {row['mean_thresh']}", axis=1
)
cum_pel_ct["norm_thresh_val"] = (
    (cum_pel_ct["threshold"] - cum_pel_ct["threshold"].min())
    / (cum_pel_ct["threshold"].max() - cum_pel_ct["threshold"].min())
).round(3)
display(cum_pel_ct)

In [ ]:
fig = go.Figure()
for id_val, id_grp in cum_pel_ct.groupby("id"):
    # Add lines by subject
    fig.add_trace(
        go.Scatter(
            x=id_grp["time"],
            y=id_grp["counter"],
            mode="lines",
            line=dict(width=2, color=subject_colors_dict[id_val]),
            name=id_val,
        )
    )
for patch_i, (patch_val, patch_grp) in enumerate(cum_pel_ct.groupby("patch_label")):
    # Add markers by patch
    fig.add_trace(
        go.Scatter(
            x=patch_grp["time"],
            y=patch_grp["counter"],
            mode="markers",
            marker=dict(
                symbol=patch_markers[patch_i],
                color=gen_hex_grad(subject_colors[-1], patch_grp["norm_thresh_val"]),
                size=8,
            ),
            name=patch_val,
            customdata=np.stack((patch_grp["threshold"],), axis=-1),
            hovertemplate="Threshold: %{customdata[0]:.2f} cm",
        )
    )

fig.update_layout(
    title="Cumulative Pellet Count per Subject", xaxis_title="Time", yaxis_title="Count"
)
fig.show()

### 5a2. Cumulative pellet count over time, per subject-patch (one line per combo)

In [ ]:
fig = go.Figure()
for id_val, id_grp in cum_pel_ct.groupby("id"):
    for patch_i, (patch_val, patch_grp) in enumerate(id_grp.groupby("patch")):
        cur_p = f"P{patch_i+1}"
        fig.add_trace(
            go.Scatter(
                x=patch_grp["time"],
                y=np.arange(1, (len(patch_grp) + 1)),
                mode="lines+markers",
                line=dict(width=2, color=subject_colors_dict[id_val]),
                marker=dict(
                    symbol=patch_markers[patch_i],
                    color=gen_hex_grad(subject_colors[-1], patch_grp["norm_thresh_val"]),
                    size=8,
                ),
                name=f"{subj} - {cur_p}: μ: {block.patch_info['mean'][patch_val]}",
                customdata=np.stack((patch_grp["threshold"],), axis=-1),
                hovertemplate="Threshold: %{customdata[0]:.2f} cm",
            )
        )

fig.update_layout(title="Cumulative Pellet Count per Subject-Patch", xaxis_title="Time", yaxis_title="Count")
fig.show()

### 5b. Pellet delivery over time, per patch, per subject

In [ ]:
fig = go.Figure()
for id_i, (id_val, id_grp) in enumerate(cum_pel_ct.groupby("id")):
    # Add lines by subject
    fig.add_trace(
        go.Scatter(
            x=id_grp["time"],
            y=id_grp["patch_label"],
            mode="lines+markers",
            line=dict(width=2, color=subject_colors_dict[id_val]),
            marker=dict(
                symbol=patch_markers[0],
                color=gen_hex_grad(subject_colors[id_i], id_grp["norm_thresh_val"]),
                size=8,
            ),
            name=id_val,
            customdata=np.stack((id_grp["threshold"],), axis=-1),
            hovertemplate="Threshold: %{customdata[0]:.2f} cm",
        )
    )


fig.update_layout(
    title="Pellet Delivery Over Time",
    xaxis_title="Time",
    yaxis_title="Patch",
    yaxis={
        "categoryorder": "array",
        "categoryarray": cum_pel_ct.sort_values("mean_thresh")[
            "patch_label"
        ].unique(),  # sort y-axis by patch threshold mean
    },
)
fig.show()

### 5c. Pellet threshold vals over time, per patch, per subject

In [ ]:
fig = go.Figure()
for id_val, id_grp in cum_pel_ct.groupby("id"):
    # Add lines by subject
    fig.add_trace(
        go.Scatter(
            x=id_grp["time"],
            y=id_grp["threshold"],
            mode="lines",
            line=dict(width=2, color=subject_colors_dict[id_val]),
            name=id_val,
        )
    )
for patch_i, (patch_val, patch_grp) in enumerate(cum_pel_ct.groupby("patch_label")):
    # Add markers by patch
    fig.add_trace(
        go.Scatter(
            x=patch_grp["time"],
            y=patch_grp["threshold"],
            mode="markers",
            marker=dict(
                symbol=patch_markers[patch_i],
                color="black",
                size=8
            ),
            name=patch_val,
        )
    )

fig.update_layout(
    title="Pellet Thresholds", xaxis_title="Time", yaxis_title="Threshold (cm)"
)
fig.show()

### 6. Cumulative wheel distance over time, per patch, per subject

In [ ]:
cum_wheel_dist = good_blocks_df["cum_wheel_dist"].iloc[b_i]
patches = block.patch_info.index

fig = go.Figure()
for subj_i, subj in enumerate(block.subjects):
    for patch_i, p in enumerate(patches):
        cur_cum_wheel_dist = cum_wheel_dist[p][subj]
        cur_p = f"P{patch_i+1}"
        fig.add_trace(
            go.Scatter(
                x=cur_cum_wheel_dist.index,
                y=cur_cum_wheel_dist,
                mode="lines",  # +  markers",
                line=dict(width=2, color=subject_colors[subj_i], dash=patch_markers_linestyles[patch_i]),
                name=f"{subj} - {cur_p}: μ: {block.patch_info['mean'][p]}",
                #legendgroup=subj,
                #showlegend=False,
            )
        )
        # Add markers for each pellet
        cur_cum_pel_ct = pd.merge_asof(
            cum_pel_ct[(cum_pel_ct["id"] == subj) &
                       (cum_pel_ct["patch"] == p)],
            cur_cum_wheel_dist.reset_index(name="cum_wheel_dist"),
            on="time",
            direction="forward",
            tolerance=pd.Timedelta("0.1s"),
        )
        if not cur_cum_pel_ct.empty:
            fig.add_trace(
                go.Scatter(
                    x=cur_cum_pel_ct["time"],
                    y=cur_cum_pel_ct["cum_wheel_dist"],
                    mode="markers",
                    marker=dict(
                        symbol=patch_markers[patch_i],
                        color=gen_hex_grad(subject_colors[-1], cur_cum_pel_ct["norm_thresh_val"]),
                        size=8,
                    ),
                    showlegend=False,
                    #name=cur_cum_pel_ct["patch_label"].iloc[0],
                    #legendgroup=subj,
                    #legendgrouptitle_text=subj,
                    customdata=np.stack(
                        (cur_cum_pel_ct["threshold"],), axis=-1),
                    hovertemplate="Threshold: %{customdata[0]:.2f} cm",
                )
            )
fig.update_layout(
    title="Cumulative Wheel Distance", xaxis_title="Time", yaxis_title="Distance (cm)"
)
fig.show()

### 7. Patch Preference: Cumulative, normalized, by wheel_distance and in_patch_time

In [ ]:
"""Using DJ."""

key = {"experiment_name": "social0.2-aeon3", "block_start": "2024-02-11 13:36:10"}
block_patches = (BlockAnalysis.Patch & key).fetch(as_dict=True)
block_subjects = (BlockAnalysis.Subject & key).fetch(as_dict=True)
subject_names = [s["subject_name"] for s in block_subjects]
patch_names = [p["patch_name"] for p in block_patches]
# Construct subject position dataframe
subjects_positions_df = pd.concat(
    [
        pd.DataFrame(
            {"subject_name": [s["subject_name"]] * len(s["position_timestamps"])}
            | {
                k: s[k]
                for k in (
                    "position_timestamps",
                    "position_x",
                    "position_y",
                    "position_likelihood",
                )
            }
        )
        for s in block_subjects
    ]
)
subjects_positions_df.set_index("position_timestamps", inplace=True)

In [ ]:
in_patch_radius = 130  # pixels
pref_attrs = ["cum_dist", "cum_time", "cum_pref_dist", "cum_pref_time"]
cols = [
    s + "_" + p + "_" + a
    for s in subject_names
    for p in patch_names
    for a in pref_attrs
]
all_subj_patch_pref = pd.DataFrame(columns=cols)
for patch in block_patches:
    cum_wheel_dist = pd.Series(
        index=patch["wheel_timestamps"], data=patch["wheel_cumsum_distance_travelled"]
    )
    # Assign pellets and wheel timestamps to subjects
    if len(block_subjects) == 1:  # TODO implement for single subject
        pass
    else:
        # Assign id based on which subject was closest to patch at time of event
        # Get distance-to-patch at each wheel ts and pel del ts, organized by subject
        # Get patch x,y from metadata patch rfid loc
        patch_center = (
            streams.RfidReader * streams.RfidReader.Attribute
            & key
            & f"rfid_reader_name LIKE '%{patch['patch_name']}%'"
            & "attribute_name = 'Location'"
        ).fetch1("attribute_value")
        patch_center = (int(patch_center["X"]), int(patch_center["Y"]))
        subjects_xy = subjects_positions_df[["position_x", "position_y"]].values
        dist_to_patch = np.sqrt(
            np.sum((subjects_xy - patch_center) ** 2, axis=1).astype(float)
        )
        dist_to_patch_df = subjects_positions_df[["subject_name"]].copy()
        dist_to_patch_df["dist_to_patch"] = dist_to_patch
        dist_to_patch_wheel_ts_id_df = pd.DataFrame(
            index=cum_wheel_dist.index, columns=subject_names
        )
        dist_to_patch_pel_ts_id_df = pd.DataFrame(
            index=patch["pellet_timestamps"], columns=subject_names
        )
        for subject_name in subject_names:
            # Find closest match between pose_df indices and wheel indices
            if not dist_to_patch_wheel_ts_id_df.empty:
                dist_to_patch_wheel_ts_subj = pd.merge_asof(
                    left=pd.DataFrame(
                        dist_to_patch_wheel_ts_id_df[subject_name].copy()
                    ).reset_index(names="time"),
                    right=dist_to_patch_df[
                        dist_to_patch_df["subject_name"] == subject_name
                    ]
                    .copy()
                    .reset_index(names="time"),
                    on="time",
                    # left_index=True,
                    # right_index=True,
                    direction="nearest",
                    tolerance=pd.Timedelta("100ms"),
                )
                dist_to_patch_wheel_ts_id_df[subject_name] = (
                    dist_to_patch_wheel_ts_subj["dist_to_patch"].values
                )
            # Find closest match between pose_df indices and pel indices
            if not dist_to_patch_pel_ts_id_df.empty:
                dist_to_patch_pel_ts_subj = pd.merge_asof(
                    left=pd.DataFrame(
                        dist_to_patch_pel_ts_id_df[subject_name].copy()
                    ).reset_index(names="time"),
                    right=dist_to_patch_df[
                        dist_to_patch_df["subject_name"] == subject_name
                    ]
                    .copy()
                    .reset_index(names="time"),
                    on="time",
                    # left_index=True,
                    # right_index=True,
                    direction="nearest",
                    tolerance=pd.Timedelta("200ms"),
                )
                dist_to_patch_pel_ts_id_df[subject_name] = dist_to_patch_pel_ts_subj[
                    "dist_to_patch"
                ].values
        # Get closest subject to patch at each wheel timestep
        cum_wheel_dist_subj_df = pd.DataFrame(
            index=cum_wheel_dist.index, columns=subject_names, data=0.0
        )
        closest_subjects = dist_to_patch_wheel_ts_id_df.idxmin(axis=1)
        wheel_dist = cum_wheel_dist.diff().fillna(cum_wheel_dist.iloc[0])
        # Assign wheel dist to closest subject for each wheel timestep
        for subject_name in subject_names:
            subj_idxs = cum_wheel_dist_subj_df[closest_subjects == subject_name].index
            cum_wheel_dist_subj_df.loc[subj_idxs, subject_name] = wheel_dist[subj_idxs]
        cum_wheel_dist_subj_df = cum_wheel_dist_subj_df.cumsum(axis=0)
        # In patch time
        in_patch = dist_to_patch_wheel_ts_id_df < in_patch_radius
        # Fill in `all_subj_patch_pref`
        for subject_name in subject_names:
            all_subj_patch_pref[
                subject_name + "_" + patch["patch_name"] + "_" + "cum_dist"
            ] = cum_wheel_dist_subj_df[subject_name].values
            dt = (in_patch.index[1] - in_patch.index[0]) / 1e6  # ms
            all_subj_patch_pref[
                subject_name + "_" + patch["patch_name"] + "_" + "cum_time"
            ] = (in_patch[subject_name].cumsum().values * dt).astype(
                int
            )  # ms
# Now that we have computed all individual patch and subject values, we iterate again through
# patches and subjects to compute preference scores
for subject_name in subject_names:
    # Get sum of subj cum wheel dists and cum in patch time
    dist_cols = [
        col
        for col in all_subj_patch_pref.columns
        if col.startswith(subject_name) and col.endswith("cum_dist")
    ]
    all_cum_dist = np.sum(all_subj_patch_pref[dist_cols].iloc[-1])
    time_cols = [
        col
        for col in all_subj_patch_pref.columns
        if col.startswith(subject_name) and col.endswith("cum_time")
    ]
    all_cum_time = np.sum(all_subj_patch_pref[time_cols].iloc[-1])
    for patch in block_patches:
        all_subj_patch_pref[
            subject_name + "_" + patch["patch_name"] + "_" + "cum_pref_dist"
        ] = (
            all_subj_patch_pref[
                subject_name + "_" + patch["patch_name"] + "_" + "cum_dist"
            ]
            / all_cum_dist
        )
        all_subj_patch_pref[
            subject_name + "_" + patch["patch_name"] + "_" + "cum_pref_time"
        ] = (
            all_subj_patch_pref[
                subject_name + "_" + patch["patch_name"] + "_" + "cum_time"
            ]
            / all_cum_time
        )

all_subj_patch_pref["time"] = cum_wheel_dist.index
display(all_subj_patch_pref)

In [ ]:
# Wheel dist pref figure

# For subject, for patch, plot cum_pref_dist

fig = go.Figure()
for subj_i, subj in enumerate(subject_names):
    for patch_i, p in enumerate(patch_names):
        cur_pref = all_subj_patch_pref[f"{subj}_{p}_cum_pref_dist"]
        cur_p = f"P{patch_i+1}"
        fig.add_trace(
            go.Scatter(
                x=all_subj_patch_pref["time"],
                y=cur_pref,
                mode="lines",  # +  markers",
                line=dict(
                    width=2,
                    color=subject_colors[subj_i],
                    dash=patch_markers_linestyles[patch_i],
                ),
                name=f"{subj} - {cur_p}: μ: {block.patch_info['mean'][p]}",
            )
        )
        # Add markers for each pellet
        cur_cum_pel_ct = pd.merge_asof(
            cum_pel_ct[(cum_pel_ct["id"] == subj) & (cum_pel_ct["patch"] == p)],
            all_subj_patch_pref,
            on="time",
            direction="forward",
            tolerance=pd.Timedelta("0.1s"),
        )
        if not cur_cum_pel_ct.empty:
            fig.add_trace(
                go.Scatter(
                    x=cur_cum_pel_ct["time"],
                    y=cur_cum_pel_ct[f"{subj}_{p}_cum_pref_dist"],
                    mode="markers",
                    marker=dict(
                        symbol=patch_markers[patch_i],
                        color=gen_hex_grad(
                            subject_colors[-1], cur_cum_pel_ct["norm_thresh_val"]
                        ),
                        size=8,
                    ),
                    showlegend=False,
                    customdata=np.stack((cur_cum_pel_ct["threshold"],), axis=-1),
                    hovertemplate="Threshold: %{customdata[0]:.2f} cm",
                )
            )
fig.update_layout(
    title="Cumulative Patch Preference - Wheel Distance",
    xaxis_title="Time",
    yaxis_title="Pref Index",
)
fig.show()

In [ ]:
# In-patch time pref figure

# For subject, for patch, plot cum_pref_time

fig = go.Figure()
for subj_i, subj in enumerate(subject_names):
    for patch_i, p in enumerate(patch_names):
        cur_pref = all_subj_patch_pref[f"{subj}_{p}_cum_pref_time"]
        cur_p = f"P{patch_i+1}"
        fig.add_trace(
            go.Scatter(
                x=all_subj_patch_pref["time"],
                y=cur_pref,
                mode="lines",  # +  markers",
                line=dict(
                    width=2,
                    color=subject_colors[subj_i],
                    dash=patch_markers_linestyles[patch_i],
                ),
                name=f"{subj} - {cur_p}: μ: {block.patch_info['mean'][p]}",
            )
        )
        # Add markers for each pellet
        cur_cum_pel_ct = pd.merge_asof(
            cum_pel_ct[(cum_pel_ct["id"] == subj) & (cum_pel_ct["patch"] == p)],
            all_subj_patch_pref,
            on="time",
            direction="forward",
            tolerance=pd.Timedelta("0.1s"),
        )
        if not cur_cum_pel_ct.empty:
            fig.add_trace(
                go.Scatter(
                    x=cur_cum_pel_ct["time"],
                    y=cur_cum_pel_ct[f"{subj}_{p}_cum_pref_time"],
                    mode="markers",
                    marker=dict(
                        symbol=patch_markers[patch_i],
                        color=gen_hex_grad(
                            subject_colors[-1], cur_cum_pel_ct["norm_thresh_val"]
                        ),
                        size=8,
                    ),
                    showlegend=False,
                    customdata=np.stack((cur_cum_pel_ct["threshold"],), axis=-1),
                    hovertemplate="Threshold: %{customdata[0]:.2f} cm",
                )
            )
fig.update_layout(
    title="Cumulative Patch Preference - Patch Time",
    xaxis_title="Time",
    yaxis_title="Pref Index",
)
fig.show()

## Overall Plots (per experiment and room)

### 1. Threshold vals per block per subject

In [ ]:
# Consider only blocks with more than 15 pellets

pel_thresh = 15
foraging_blocks = good_blocks_df_cp[
    good_blocks_df_cp["pellet_info"].apply(len) > pel_thresh
].reset_index(drop=True)

In [ ]:
# Create column "block_type" with values ["presocial", "social", "postsocial"]

mask_social = foraging_blocks["subjects"].apply(len) > 1
subj_str = foraging_blocks["subjects"].apply(str)
presocial_condition = foraging_blocks[mask_social].groupby(subj_str)[
    "start"].min()
postsocial_condition = foraging_blocks[mask_social].groupby(subj_str)[
    "end"].max()


def assign_block_type(row):
    if len(row["subjects"]) > 1:
        return "social"
    else:
        subj = row["subjects"][0]
        idx = next(
            iter([idx for idx in presocial_condition.index if subj in idx]), None
        )
        if idx is not None:
            if row["start"] < presocial_condition[idx]:
                return "presocial"
            elif row["start"] > postsocial_condition[idx]:
                return "postsocial"


foraging_blocks["block_type"] = foraging_blocks.apply(
    assign_block_type, axis=1)

In [ ]:
# Threshold vals obtained from halfway thru block (lower better, obvs)

fig = go.Figure()
legend_added = set()  # legend only for unique subjs
learning_threshold_vals = pd.DataFrame(columns=["threshold_vals", "id"])
block_type_markers = {
    "presocial": patch_markers[0],
    "social": patch_markers[1],
    "postsocial": patch_markers[2],
}
for block_i, block in enumerate(foraging_blocks.itertuples()):
    n_pellets = len(block.pellet_info)
    pellet_info = block.pellet_info.sort_values("time")
    pellet_info_post = pellet_info.iloc[
        (n_pellets // 2) :
    ]  # halfway point (in terms of pellets obtained)
    # Get threshold vals, per subject.
    for subj_val, subj_grp in pellet_info_post.groupby("id"):
        # Add threshold_vals, id as row to `learning_threshold_vals`.
        new_row = pd.DataFrame(columns=["threshold_vals", "id"], index=[block.Index])
        new_row["threshold_vals"], new_row["id"] = (
            [subj_grp["threshold"].values],
            subj_val,
        )
        learning_threshold_vals = pd.concat((learning_threshold_vals, new_row))
        # subj_val
        grouping_factor = f"{patch_markers_dict.get(block_type_markers[block.block_type])} {
            subj_val} {block.block_type}"
        show_in_legend = grouping_factor not in legend_added
        # If not, add it and set the flag to False for subsequent rows with the same ID
        if show_in_legend:
            legend_added.add(grouping_factor)
        fig.add_trace(
            go.Box(
                y=new_row.loc[block.Index, "threshold_vals"],
                x=[block.Index] * len(new_row.loc[block.Index, "threshold_vals"]),
                name=grouping_factor,
                legendgroup=grouping_factor,
                boxpoints="all",
                # jitter=0.5,
                marker={
                    "color": subject_colors_dict[subj_val],
                    "symbol": block_type_markers[block.block_type],
                },
                showlegend=show_in_legend,  # legend only for unique subjs
            )
        )

fig.update_layout(
    title="Pellet Thresholds",
    xaxis_title="Block Index",
    yaxis_title="Threshold (cm)",
)
fig.update_layout(legend_tracegroupgap=0)
fig.show()

### 2. Patch preference per block per subject

In [ ]:
# Patch preference by wheel dist spun from halfway through block

pp_overall = pd.DataFrame(
    columns=["one_best_patch", "two_best_patches", "id", "block_type"]
)  # e.g. easy vs. (medium + hard); (easy + medium) vs. hard

for block_i, block in enumerate(foraging_blocks.itertuples()):
    easy_p, med_p, hard_p = block.patch_info.sort_values("mean").index
    n_pellets = len(block.pellet_info)
    pellet_info = block.pellet_info.sort_values("time")
    halfway_ts = pellet_info["time"].iloc[n_pellets // 2]
    cum_wheel_dist = good_blocks_df["cum_wheel_dist"].loc[block.Index]
    for subj in block.subjects:
        easy_dist = cum_wheel_dist[easy_p][subj]
        easy_dist = (
            easy_dist[easy_dist.index > halfway_ts].iloc[-1]
            - easy_dist[easy_dist.index > halfway_ts].iloc[0]
        )
        easy_dist = 1 if easy_dist < 0 else easy_dist
        med_dist = cum_wheel_dist[med_p][subj]
        med_dist = (
            med_dist[med_dist.index > halfway_ts].iloc[-1]
            - med_dist[med_dist.index > halfway_ts].iloc[0]
        )
        med_dist = 1 if med_dist < 0 else med_dist
        hard_dist = cum_wheel_dist[hard_p][subj]
        hard_dist = (
            hard_dist[hard_dist.index > halfway_ts].iloc[-1]
            - hard_dist[hard_dist.index > halfway_ts].iloc[0]
        )
        hard_dist = 1 if hard_dist < 0 else hard_dist
        new_row = pd.DataFrame(
            columns=["one_best_patch", "two_best_patches"], index=[block.Index]
        )
        new_row["one_best_patch"] = np.round(
            (easy_dist / (easy_dist + med_dist + hard_dist)), 3
        )
        new_row["two_best_patches"] = np.round(
            ((easy_dist + med_dist) / (easy_dist + med_dist + hard_dist)), 3
        )
        new_row["id"] = subj
        new_row["block_type"] = block.block_type
        pp_overall = pd.concat((pp_overall, new_row))

In [ ]:
fig = go.Figure()
for block_type in block_type_markers.keys():
    for id, group in pp_overall[pp_overall["block_type"] == block_type].groupby("id"):
        fig.add_trace(
            go.Scatter(
                x=group.index,
                y=group["one_best_patch"],
                mode="markers",
                name=block_type,
                legendgroup=id,
                legendgrouptitle_text=id,
                marker={"color": subject_colors_dict[id]},
                marker_symbol=block_type_markers[block_type],
            )
        )
fig.update_layout(
    title="Patch Preference: Easy Only",
    xaxis_title="Block Index",
    yaxis_title="Preference",
    legend_tracegroupgap=0,
)
fig.update_traces(
    legendgrouptitle_font_size=12,
)
fig.show()

In [ ]:
fig = px.scatter(
    pp_overall,
    x=pp_overall.index,
    y=["one_best_patch"],
    color="id",
    symbol="block_type",
    title="Patch Preference: Easy Only",
    labels={"index": "Block Index", "value": "Preference"},
)
fig.update_layout(legend_title_text="")
fig.for_each_trace(lambda trace: setattr(
    trace, "name", trace.name.replace(",", "")))
fig.show()

In [ ]:
fig = px.scatter(
    pp_overall,
    x=pp_overall.index,
    y=["two_best_patches"],
    color="id",
    symbol="block_type",
    title="Patch Preference: Easy + Medium",
    labels={"index": "Block Index", "value": "Preference"},
)
fig.update_layout(legend_title_text="")
fig.for_each_trace(lambda trace: setattr(
    trace, "name", trace.name.replace(",", "")))
fig.show()

### 2b. Patch Preference by Block Type (presocial, social, postsocial)

In [ ]:
fig = px.box(
    pp_overall,
    x="block_type",
    y="one_best_patch",
    color="id",
    points="all",
    title="Patch Preference: Easy Only by Block Type",
    labels={"block_type": "Block Type", "one_best_patch": "Preference"},
)
fig.update_layout(legend_title_text="ID")
fig.show()

In [ ]:
fig = px.box(
    pp_overall,
    x="block_type",
    y="two_best_patches",
    color="id",
    points="all",
    title="Patch Preference: Easy + Medium by Block Type",
    labels={"block_type": "Block Type", "two_best_patches": "Preference"},
)
fig.update_layout(legend_title_text="ID")
fig.show()

### 2c. Patch Preference Difference between First- and Second-Half of Block

### 3a. Cumulative Wheel Distance by Patch Difficulty

### 3b. Cumulative Wheel Distance by Patch ID

---

## (WIP) Correct ID swaps

In [ ]:
# There are times when the same ID is assigned to multiple animals in the same frame
# Try to assign the row with duplicated ID with lower likelihood to another ID
centroid_df_cp = centroid_df.reset_index().copy()
classes = np.array(centroid_df_cp["class"].unique())
# Mask for rows with multiple assignments of the same ID at the same time
many_to_one_mask = centroid_df_cp.groupby(["time", "class"]).transform("size") > 1
duplicated_data = centroid_df_cp.loc[many_to_one_mask]
# Indices for rows with lower likelihood
low_likelihood_idx = duplicated_data.loc[
    ~duplicated_data.index.isin(
        duplicated_data.groupby(["time", "class"])["class_likelihood"].idxmax()
    )
].index
# This assigns another class randomly (in 2-animal case, it's the other animal, but in >2-animal case, it may assign duplicate IDs again)
centroid_df_cp.loc[low_likelihood_idx, "class"] = centroid_df_cp.loc[
    low_likelihood_idx
].apply(lambda x: np.random.choice(classes[classes != x["class"]]), axis=1)
centroid_df_cp.set_index("time", inplace=True)

In [ ]:
# Try on a smaller subset of data
centroid_df_cp = centroid_df_cp.loc["2024-02-18 11:00:00":"2024-02-18 11:05:00"].copy()
# Compute instantaneous speed
centroid_df_cp["speed"] = (
    centroid_df_cp.groupby("class")[["x", "y"]].diff().apply(
        np.linalg.norm, axis=1)
    / centroid_df_cp.reset_index()
    .groupby("class")["time"]
    .diff()
    .dt.total_seconds()
    .values
)
speed_threshold = 500
classes = centroid_df_cp["class"].unique()
timestamps = centroid_df_cp.index.unique()  # assuming timestamps are sorted
speed_mask = (np.isfinite(centroid_df_cp["speed"].values)) & (
    centroid_df_cp["speed"] > speed_threshold
)
dtypes_dict = centroid_df_cp.dtypes.to_dict()

In [ ]:
from scipy.optimize import linear_sum_assignment

while speed_mask.any():  # while there are speed violations
    # work on "windows" of 2 consecutive violations
    first_two_violations = centroid_df_cp[speed_mask].index.unique().sort_values()[
        :2]
    if len(first_two_violations) > 1:
        start_window, end_window = pd.to_datetime(first_two_violations)
        swap_window = centroid_df_cp.loc[start_window:end_window]
    else:  # last violation
        start_window = first_two_violations.iloc[0]
        swap_window = centroid_df_cp.loc[start_window:]
    for curr_timestamp in swap_window.index.unique():
        # use previous rows to correct id assignment based on speed
        prev_timestamp = timestamps[np.where(
            timestamps == curr_timestamp)[0][0] - 1]
        prev_rows = centroid_df_cp.loc[prev_timestamp]
        curr_rows = centroid_df_cp.loc[curr_timestamp]
        if isinstance(prev_rows, pd.Series):
            prev_rows = prev_rows.to_frame().T.astype(dtypes_dict)
        if isinstance(curr_rows, pd.Series):
            curr_rows = curr_rows.to_frame().T.astype(dtypes_dict)
        prev_rows = prev_rows.reset_index().rename(columns={"index": "time"})
        curr_rows = curr_rows.reset_index().rename(columns={"index": "time"})
        # initialise np array to store the hungarian cost matrix
        cost_matrix = np.zeros((len(prev_rows), len(curr_rows)))
        for prev_row in prev_rows.itertuples():
            for curr_row in curr_rows.itertuples():
                # Calculate speed
                x_diff = curr_row.x - prev_row.x
                y_diff = curr_row.y - prev_row.y
                distance = np.sqrt(x_diff**2 + y_diff**2)
                time_diff = (curr_row.time - prev_row.time).total_seconds()
                cost_matrix[prev_row.Index, curr_row.Index] = (
                    distance / time_diff if time_diff != 0 else np.nan
                )
        row_ind, col_ind = linear_sum_assignment(cost_matrix)
        if len(prev_rows) < len(curr_rows):
            classes_to_assign = classes.copy()
            swap_order = {
                prev_rows.loc[r, "class"]: c for r, c in zip(row_ind, col_ind)
            }
            for id, new in swap_order.items():
                # get the index of ID
                old = np.where(classes_to_assign == id)[0][0]
                if old != new:
                    # swap the values
                    classes_to_assign[new], classes_to_assign[old] = (
                        classes_to_assign[old],
                        classes_to_assign[new],
                    )
            centroid_df_cp.loc[curr_timestamp, "class"] = classes_to_assign[
                : len(curr_rows)
            ]
        else:
            curr_rows.loc[col_ind,
                          "class"] = prev_rows.loc[row_ind, "class"].values
            curr_rows = curr_rows.loc[col_ind].set_index("time", drop=True)
            centroid_df_cp.loc[curr_timestamp, "class"] = prev_rows.loc[
                row_ind, "class"
            ].values
    # recompute speed and speed_mask
    centroid_df_cp["speed"] = (
        centroid_df_cp.groupby("class")[["x", "y"]].diff().apply(
            np.linalg.norm, axis=1)
        / centroid_df_cp.reset_index()
        .groupby("class")["time"]
        .diff()
        .dt.total_seconds()
        .values
    )
    speed_mask = (np.isfinite(centroid_df_cp["speed"].values)) & (
        centroid_df_cp["speed"] > speed_threshold
    )

In [ ]:
# Compute final speed
centroid_df_cp["speed"] = (
    centroid_df_cp.groupby("class")[["x", "y"]].diff().apply(np.linalg.norm, axis=1)
    / centroid_df_cp.reset_index()
    .groupby("class")["time"]
    .diff()
    .dt.total_seconds()
    .values
)

In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
data = centroid_df_cp.loc["2024-02-18 11:00:00":"2024-02-18 11:01:31.860000134"]
classes = data["class"].unique()
for class_ in classes:
    fig.append_trace(
        go.Scatter(
            x=data[data["class"] == class_].index,
            y=data[data["class"] == class_]["x"],
            mode="markers",
            name=str(class_),  # Use the class as the name of the trace
            marker=dict(color=subject_colors_dict[class_]),
            hovertemplate="Speed: %{text}",
            text=data[data["class"] == class_]["speed"].tolist(),
        ),
        row=1,
        col=1,
    )
    fig.append_trace(
        go.Scatter(
            x=data[data["class"] == class_].index,
            y=data[data["class"] == class_]["y"],
            mode="markers",
            name=str(class_),  # Use the class as the name of the trace
            marker=dict(color=subject_colors_dict[class_]),
            hovertemplate="Speed: %{text}",
            text=data[data["class"] == class_]["speed"].tolist(),
        ),
        row=2,
        col=1,
    )
fig.update_yaxes(title_text="x position", row=1, col=1)
fig.update_yaxes(title_text="y position", row=2, col=1)

fig.show()

In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
data = centroid_df.loc["2024-02-18 11:00:00":"2024-02-18 11:01:31.860000134"].copy()
# Compute speed
data["speed"] = (
    data.groupby("class")[["x", "y"]].diff().apply(np.linalg.norm, axis=1)
    / data.reset_index().groupby("class")["time"].diff().dt.total_seconds().values
)
classes = data["class"].unique()
for class_ in classes:
    fig.append_trace(
        go.Scatter(
            x=data[data["class"] == class_].index,
            y=data[data["class"] == class_]["x"],
            mode="markers",
            name=str(class_),  # Use the class as the name of the trace
            marker=dict(color=subject_colors_dict[class_]),
            hovertemplate="Speed: %{text}",
            text=data[data["class"] == class_]["speed"].tolist(),
        ),
        row=1,
        col=1,
    )
    fig.append_trace(
        go.Scatter(
            x=data[data["class"] == class_].index,
            y=data[data["class"] == class_]["y"],
            mode="markers",
            name=str(class_),  # Use the class as the name of the trace
            marker=dict(color=subject_colors_dict[class_]),
            hovertemplate="Speed: %{text}",
            text=data[data["class"] == class_]["speed"].tolist(),
        ),
        row=2,
        col=1,
    )
fig.update_yaxes(title_text="x position", row=1, col=1)
fig.update_yaxes(title_text="y position", row=2, col=1)

fig.show()

In [ ]:
"""Functions to retrieve and show single frame. """

import os

from aeon.io.video import frames


def get_single_frame(
    root: str | os.PathLike,
    video_reader: aeon.io.reader.Video,
    time: pd.Timestamp,
) -> np.ndarray:
    """
    Retrieve a single frame from the given root directory,
    Video reader, and time.

    Args:
        root (str or os.PathLike): The root path where epoch data
            is stored.
        video_reader (aeon.io.reader.Video): The Video reader.
        time (pd.Timestamp): The timestamp of the frame to retrieve.

    Returns:
        numpy.ndarray: The raw frame.
    """
    vdata = aeon.load(
        root, video_reader, start=time, end=time + pd.Timedelta(seconds=1)
    )
    vframe = frames(vdata.iloc[:1])
    return np.squeeze(list(vframe))


def plot_overlay(
    img: np.ndarray,
    pose: pd.DataFrame = None,
    roi_coords: "list[list[tuple[int, int]]]" = None,
) -> None:
    """
    Overlay any pose data and/or ROI points on a frame (image).

    Args:
        img (numpy.ndarray): The raw frame.
        pose (pandas.DataFrame): The pose data with "class" as a grouping column.
        roi_coords (list[list[tuple[int, int]]]): A list of lists of tuples
            representing the ROI x, y coordinates.

    Returns:
        None
    """
    fig = px.imshow(img)
    if pose is not None:
        scatter = px.scatter(
            pose,
            x="x",
            y="y",
            color="class",
            opacity=0.8,
        )
        for trace in scatter.data:
            fig.add_trace(trace)
    if roi_coords is not None:
        for roi_coord in roi_coords:
            pts = roi_coord + [roi_coord[0]]
            xs, ys = zip(*pts)
            fig.add_scatter(x=xs, y=ys, fill="toself")
    fig.show()

In [ ]:
img = get_single_frame(
    roots[0], social02.CameraTop.Video, pd.Timestamp("2024-02-18 11:00:00")
)
plot_overlay(
    img, pose=centroid_df_cp.loc["2024-02-18 11:00:00":"2024-02-18 11:01:31.860000134"]
)